Лабораторна робота №7 ФІТ 4-6 Піць А.В.
Завдання №1

In [1]:
# Завантажуємо необхідні бібліотеки
!pip install scikit-surprise

from surprise import Dataset, KNNBasic, SVD
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split
import pandas as pd

# Завантаження датасету
data = Dataset.load_builtin('ml-100k')

# Виведення перших 5 рядків
raw_ratings = data.raw_ratings[:5]
for row in raw_ratings:
    print(row)

# Розбиваємо датасет на тренувальну і тестову вибірки
trainset, testset = train_test_split(data, test_size=0.2)

# Алгоритм 1: KNNBasic
knn = KNNBasic()
knn.fit(trainset)
predictions_knn = knn.test(testset)
mae_knn = accuracy.mae(predictions_knn)

# Алгоритм 2: SVD
svd = SVD()
svd.fit(trainset)
predictions_svd = svd.test(testset)
mae_svd = accuracy.mae(predictions_svd)

# Вибір найкращого алгоритму на основі MAE
best_algo = 'SVD' if mae_svd < mae_knn else 'KNNBasic'
print(f'Найкращий алгоритм: {best_algo}')

# Крос-валідація для підбору параметрів для SVD
param_grid = {'n_epochs': [20, 30], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Виведення найкращих параметрів
print(gs.best_params['mae'])

# Рекомендація для конкретного користувача
algo = gs.best_estimator['mae']
algo.fit(trainset)

# Виведення 10 фільмів для користувача з id 196
user_id = '196'
user_ratings = data.build_full_trainset().ur[trainset.to_inner_uid(user_id)]
user_watched_movies = [trainset.to_raw_iid(iid) for (iid, _) in user_ratings]

recommendations = []
for iid in algo.trainset.all_items():
    raw_iid = algo.trainset.to_raw_iid(iid)
    if raw_iid not in user_watched_movies:
        pred = algo.predict(user_id, raw_iid)
        recommendations.append((raw_iid, pred.est))

recommendations.sort(key=lambda x: x[1], reverse=True)
print("Рекомендовані фільми для користувача 196:")
for movie_id, score in recommendations[:10]:
    print(f"Фільм {movie_id}, Оцінка {score:.2f}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357276 sha256=27816881aa626a58f857d639166bc10db6d466a41b707337ab6e397ed26aac45
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise
Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')
Computing the msd similarity matrix...
Done computing similar

Висновок:Ми завантажили датасет рецензій (ml-100k) за допомогою бібліотеки Surprise і провели аналіз перших 5 рядків для розуміння структури даних.
Ми реалізували два алгоритми: KNNBasic і SVD, для рекомендаційної системи. Після навчання моделей на тренувальних даних і оцінки за допомогою тестових даних, було визначено, що алгоритм SVD має меншу похибку MAE.
Після підбору параметрів за допомогою крос-валідації для алгоритму SVD, ми отримали найкращі параметри для моделі. Використовуючи ці параметри, було зроблено 10 рекомендацій для конкретного користувача.

Завдання №2

In [3]:
# Завантажуємо необхідні бібліотеки
!pip install scikit-surprise

import os
import urllib.request
from surprise import Dataset, Reader, BaselineOnly
from surprise.model_selection import cross_validate

# Завантаження датасету з інтернету
if not os.path.exists('ml-100k'):
    os.makedirs('ml-100k')

url = 'https://files.grouplens.org/datasets/movielens/ml-100k/u.data'
urllib.request.urlretrieve(url, 'ml-100k/u.data')

# Вказуємо формат даних
file_path = 'ml-100k/u.data'
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file(file_path, reader=reader)

# Використання алгоритму Baseline для власної рекомендаційної системи
bsl_options = {'method': 'als', 'n_epochs': 5, 'reg_u': 12, 'reg_i': 5}
algo = BaselineOnly(bsl_options=bsl_options)

# Крос-валідація для оцінки алгоритму
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Отримання рекомендацій для конкретного користувача
trainset = data.build_full_trainset()
algo.fit(trainset)

user_id = '196'  # Користувач, для якого отримаємо рекомендації
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
user_watched_movies = [trainset.to_raw_iid(iid) for (iid, _) in user_ratings]

recommendations = []
for iid in algo.trainset.all_items():
    raw_iid = algo.trainset.to_raw_iid(iid)
    if raw_iid not in user_watched_movies:
        pred = algo.predict(user_id, raw_iid)
        recommendations.append((raw_iid, pred.est))

recommendations.sort(key=lambda x: x[1], reverse=True)
print("Рекомендовані фільми для користувача 196:")
for movie_id, score in recommendations[:10]:
    print(f"Фільм {movie_id}, Оцінка {score:.2f}")



Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9391  0.9347  0.9400  0.9505  0.9426  0.9414  0.0052  
MAE (testset)     0.7440  0.7397  0.7463  0.7521  0.7450  0.7454  0.0040  
Fit time          0.19    0.22    0.22    0.34    0.23    0.24    0.05    
Test time         0.09    0.09    0.24    0.11    0.09    0.12    0.06    
Estimating biases using als...
Рекомендовані фільми для користувача 196:
Фільм 408, Оцінка 4.51
Фільм 318, Оцінка 4.47
Фільм 483, Оцінка 4.47
Фільм 169, Оцінка 4.47
Фільм 64, Оцінка 4.45
Фільм 12, Оцінка 4.39
Фільм 114, Оцінка 4.39
Фільм 603, Оцінка 4.38
Фільм 1449, Оцінка 4.35
Фільм 178, Оцінка 4.35


Висновок:Ми завантажили датасет ml-100k безпосередньо з інтернету, щоб забезпечити доступ до даних у середовищі Google Colab.
Використовуючи алгоритм BaselineOnly, ми побудували власну рекомендаційну систему, яка оцінює фільми для користувачів на основі їхніх попередніх оцінок.
За допомогою крос-валідації були оцінені результати роботи системи за метриками RMSE і MAE, і було згенеровано список рекомендованих фільмів для конкретного користувача.